# **Put Whole Document into Prompt and Ask the Model**


## Overview
In recent years, the development of Large Language Models (LLMs) like GPT-3 and GPT-4 has revolutionized the field of natural language processing (NLP). These models are capable of performing a wide range of tasks, from generating coherent text to answering questions and summarizing information. Their effectiveness, however, is not without limitations. One significant constraint is the context window length, which affects how much information can be processed at once. LLMs operate within a fixed context window, measured in tokens, with GPT-3 having a limit of 4096 tokens and GPT-4 extending to 8192 tokens. When dealing with lengthy documents, attempting to input the entire text into the model's prompt can lead to truncation, where essential information is lost, and increased computational costs due to the processing of large inputs.

These limitations become particularly pronounced when creating a retrieval-based question-answering (QA) assistant. The context length constraint restricts the ability to input all content into the prompt simultaneously, leading to potential loss of critical context and details. This necessitates the development of sophisticated strategies for selectively retrieving and processing relevant sections of the document. Techniques such as chunking the document into manageable parts, employing summarization methods, and using external retrieval systems are crucial to address these challenges. Understanding and mitigating these limitations are essential for designing effective QA systems that leverage the full potential of LLMs while navigating their inherent constraints.


## __Table of Contents__

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-required-libraries">Installing required libraries</a></li>
            <li><a href="#Importing-required-libraries">Importing required libraries</a></li>
        </ol>
    </li>
    <li><a href="#Build-LLM">Build LLM</a></li>
    <li><a href="#Load-source-document">Load source document</a></li>
    <li>
        <a href="#Limitation-of-retrieve-directly-from-full-document">Limitation of retrieve directly from full document</a>
        <ol>
            <li><a href="#Context-length">Context length</a></li>
            <li><a href="#LangChain-prompt-template">LangChain prompt template</a></li>
            <li><a href="#Use-mixtral-model">Use mixtral model</a></li>
            <li><a href="#Use-Llama-3-model">Use Llama 3 model</a></li>
            <li><a href="#Use-one-piece-of-information">Use one piece of information</a></li>
        </ol>
    </li>
</ol>

<a href="#Exercises">Exercises</a>
<ol>
    <li><a href="#Exercise-1---Change-to-use-another-LLM">Exercise 1 - Change to use another LLM</a></li>
</ol>


## Objectives

After completing this lab you will be able to:

 - Explain the concept of context length for LLMs.
 - Recognize the limitations of retrieving information when inputting the entire content of a document into a prompt.


----


## Setup


For this lab, you will use the following libraries:

*   [`ibm-watson-ai`](https://ibm.github.io/watson-machine-learning-sdk/index.html) for using LLMs from IBM's watsonx.ai.
*   [`langchain`, `langchain-ibm`, `langchain-community`](https://www.langchain.com/) for using relevant features from LangChain.


### Installing required libraries

The following required libraries are __not__ preinstalled in the Skills Network Labs environment. __You must run the following cell__ to install them:

**Note:** The version is being pinned here to specify the version. It's recommended that you do this as well. Even if the library is updated in the future, the installed library could still support this lab work.

This might take approximately 1 minute. 

As `%%capture` is used to capture the installation, you won't see the output process. After the installation is completed, you will see a number beside the cell.


In [1]:
#%%capture
#After executing the cell,please RESTART the kernel and run all the cells.
!pip install --user "ibm-watsonx-ai==1.0.10"
!pip install --user "langchain==0.2.6" 
!pip install --user "langchain-ibm==0.1.8"
!pip install --user "langchain-community==0.2.1"

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 936.7/936.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 96.0 MB/s eta 0:00:00
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.13.6-py3-none-any.whl size=77287 sha256=8dad74a612d5379113592eeded76e00829cb699bd3e876cf3340ca3be761834c
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/17/c5/52/64bd703cf3cc4a83ba0e39d9b3eedc3a15afced69bbe6cf306
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.13.6-py3-none-any.whl size=661517 sha256=d85555bef4ca5c00845202ce145b0083f36be0a9045e0068cf2e9174e50e4bc4
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/3f/11/58/646f89b99861739fce853f8776e5037d0f704d5d9623a07f7c
  Created wheel for ibm-cos-sdk-s3transfer: filename=ibm_cos_sdk_s3tr

### Importing required libraries


In [1]:
# You can use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.document_loaders import TextLoader
from langchain_ibm import WatsonxLLM

## Build LLM


Here, you will create a function that interacts with the watsonx.ai API, enabling you to utilize various models available.

You just need to input the model ID in string format, then it will return you with the LLM object. You can use it to invoke any queries. A list of model IDs can be found in [here](https://ibm.github.io/watsonx-ai-python-sdk/fm_model.html).


In [2]:
def llm_model(model_id):
    parameters = {
        GenParams.MAX_NEW_TOKENS: 256,  # this controls the maximum number of tokens in the generated output
        GenParams.TEMPERATURE: 0.5, # this randomness or creativity of the model's responses
    }
    
    credentials = {
        "url": "https://us-south.ml.cloud.ibm.com"
    }
    
    project_id = "skills-network"
    
    model = ModelInference(
        model_id=model_id,
        params=parameters,
        credentials=credentials,
        project_id=project_id
    )
    
    llm = WatsonxLLM(watsonx_model = model)
    return llm

Let's try to invoke an example query.


In [3]:
llama_llm = llm_model('meta-llama/llama-3-3-70b-instruct')

In [4]:
llama_llm.invoke("How are you?")

" How was your week? I hope it was amazing and that you got to do some amazing things.\nMy week was pretty good. I had a lot of work to do, but I also got to do some fun things. On Wednesday, I went to the park with my friends and we had a picnic. It was really nice to get out of the house and enjoy the sunshine. We brought a lot of food and drinks and just relaxed in the park. It was a great way to spend the afternoon.\nOn Thursday, I had a meeting at work and we discussed some new projects that we are going to be working on. I'm really excited about them and I think they are going to be a lot of fun. We also had a team lunch, which was nice. It's always great to catch up with my coworkers and hear about what they are working on.\nOn Friday, I had a bit of a lazy day. I didn't have any plans, so I just stayed home and watched some TV. I also did some laundry and cleaned my apartment a bit. It was nice to just relax and not have to do anything too strenuous.\nOn Saturday, I went to the

## Load source document


A document has been prepared here.


In [5]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/d_ahNwb1L2duIxBR6RD63Q/state-of-the-union.txt"

--2025-10-02 12:13:41--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/d_ahNwb1L2duIxBR6RD63Q/state-of-the-union.txt
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
200 OKequest sent, awaiting response... 
Length: 39027 (38K) [text/plain]
Saving to: ‘state-of-the-union.txt’

state-of-the-union. 100%[===================>]  38.11K  --.-KB/s    in 0.001s  

2025-10-02 12:13:41 (30.4 MB/s) - ‘state-of-the-union.txt’ saved [39027/39027]



Use `TextLoader` to load the text.


In [6]:
loader = TextLoader("state-of-the-union.txt")

In [7]:
data = loader.load()

Let's take a look at the document.


In [8]:
content = data[0].page_content
content

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with 

## Limitation of retrieve directly from full document


### Context length


Before you explore the limitations of directly retrieving information from a full document, you need to understand a concept called `context length`. 

`Context length` in LLMs refers to the amount of text or information (prompt) that the model can consider when processing or generating output. LLMs have a fixed context length, meaning they can only take into account a limited amount of text at a time.



So, how long is your source document here? The answer is 8,235 tokens, which you calculated using this [platform](https://platform.openai.com/tokenizer).


### LangChain prompt template


A prompt template has been set up using LangChain to make it reusable.

In this template, you will define two input variables:
- `content`: This variable will hold all the content from the entire source document at once.
- `question`: This variable will capture the user's query.


In [9]:
template = """According to the document content here 
            {content},
            answer this question 
            {question}.
            Do not try to make up the answer.
                
            YOUR RESPONSE:
"""

prompt_template = PromptTemplate(template=template, input_variables=['content', 'question'])
prompt_template 

PromptTemplate(input_variables=['content', 'question'], template='According to the document content here \n            {content},\n            answer this question \n            {question}.\n            Do not try to make up the answer.\n                \n            YOUR RESPONSE:\n')

### Use mixtral model


Since the context window length of the mixtral model is longer than your source document, you can assume it can retrieve relevant information for the query when you input the whole document into the prompt.


First, let's build a mixtral model.


In [11]:
#mixtral_llm = llm_model('mistralai/mixtral-8x7b-instruct-v01')
mixtral_llm = llm_model('mistralai/mistral-small-3-1-24b-instruct-2503')

Then, create a query chain.


In [12]:
query_chain = LLMChain(llm=mixtral_llm, prompt=prompt_template)

Then, set the query and get the answer.


In [13]:
query = "It is in which year of our nation?"
response = query_chain.invoke(input={'content': content, 'question': query})
print(response['text'])

            245th year


Ypu have asked a question whose answer appears at the very end of the document. Despite this, the LLM was still able to answer it correctly because the model's context window is long enough to accommodate the entire content of the document.


### Use Llama 3 model


Now, let's try using Llama3 model.


First, create a query chain.


In [14]:
query_chain = LLMChain(llm=llama_llm, prompt=prompt_template)
query_chain 

LLMChain(prompt=PromptTemplate(input_variables=['content', 'question'], template='According to the document content here \n            {content},\n            answer this question \n            {question}.\n            Do not try to make up the answer.\n                \n            YOUR RESPONSE:\n'), llm=WatsonxLLM(model_id='meta-llama/llama-3-3-70b-instruct', deployment_id=None, project_id='skills-network', space_id=None, params={'max_new_tokens': 256, 'temperature': 0.5}, watsonx_model=<ibm_watsonx_ai.foundation_models.inference.model_inference.ModelInference object at 0x7338af9f6e70>))

Then, use the query chain (the code is shown below) to invoke the LLM, which will answer the same query as before based on the entire document's content.


In [15]:
query = "It is in which year of our nation?"
response = query_chain.invoke(input={'content': content, 'question': query})
print(response['text'])

It is in our 245th year as a nation.


Now you can see It can also provide the correct answer.


#### Take away


If the document is much longer than the LLM's context length, it is important and necessary to cut the document into chunks, index them, and then let the LLM retrieve the relevant information accurately and efficiently.

In the next lesson, you will learn how to perform these operations using LangChain.


# Exercises


### Exercise 1 - Change to use another LLM


Try to use another LLM to see if error occurs. For example, try using `'ibm/granite-3-8b-instruct'`.


In [16]:
granite_llm = llm_model('ibm/granite-3-8b-instruct')
query_chain = LLMChain(llm=granite_llm, prompt=prompt_template)
query = "It is in which year of our nation?"
response = query_chain.invoke(input={'content': content, 'question': query})
print(response['text'])


The State of the Union address was delivered in the 245th year of the United States.
